### Creation of Docker file

In [ ]:
import os
from bedrock_agentcore_starter_toolkit import Runtime

# 1. Clear any old AWS credentials. There are times I'm having access issues due to expired creds, so by default i clear it as a best practice.
os.environ.pop("AWS_ACCESS_KEY_ID", None)
os.environ.pop("AWS_SECRET_ACCESS_KEY", None)
os.environ.pop("AWS_SESSION_TOKEN", None)

# 2. Set your SSO profile, here I'm utilizing aws sso login
os.environ["AWS_PROFILE"] = "sandbox"  


# 4. Initialize Runtime
agentcore_runtime = Runtime()

# 5. Configure the agent
agent_name = "main_agent"
response = agentcore_runtime.configure(
    entrypoint="main_agent.py", 
    auto_create_execution_role=True,
    auto_create_ecr=True,
    requirements_file="requirements.txt",
    region="us-east-1",
    agent_name=agent_name
)

print("AgentCore configuration response:")
print(response)

Entrypoint parsed: file=C:\Users\machupichu\Repositories\agentic-chatbot\main_agent.py, bedrock_agentcore_name=main_agent
Configuring BedrockAgentCore agent: main_agent


⚠️  ℹ️  No container engine found (Docker/Finch/Podman not installed)
✅ Default deployment uses CodeBuild (no container engine needed)
💡 Run 'agentcore launch' for cloud-based building and deployment
💡 For local builds, install Docker, Finch, or Podman

⚠️  [WARNING] Platform mismatch: Current system is 'linux/amd64' but Bedrock AgentCore requires 'linux/arm64'.
For deployment options and workarounds, see: 
https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/getting-started-custom.html

Generated Dockerfile: c:\Users\machupichu\Repositories\agentic-chatbot\Dockerfile
Generated .dockerignore: c:\Users\machupichu\Repositories\agentic-chatbot\.dockerignore
Keeping 'main_agent' as default agent
Bedrock AgentCore configured: c:\Users\machupichu\Repositories\agentic-chatbot\.bedrock_agentcore.yaml


AgentCore configuration response:
config_path=WindowsPath('c:/Users/machupichu/Repositories/agentic-chatbot/.bedrock_agentcore.yaml') dockerfile_path=WindowsPath('c:/Users/machupichu/Repositories/agentic-chatbot/Dockerfile') dockerignore_path=WindowsPath('c:/Users/machupichu/Repositories/agentic-chatbot/.dockerignore') runtime='None' region='us-east-1' account_id='943986266082' execution_role=None ecr_repository=None auto_create_ecr=True


# Deploy the Agent

In [10]:
launch_result = agentcore_runtime.launch()

🚀 CodeBuild mode: building in cloud (RECOMMENDED - DEFAULT)
   • Build ARM64 containers in the cloud with CodeBuild
   • No local Docker required
💡 Available deployment modes:
   • runtime.launch()                           → CodeBuild (current)
   • runtime.launch(local=True)                 → Local development
   • runtime.launch(local_build=True)           → Local build + cloud deploy (NEW)
Starting CodeBuild ARM64 deployment for agent 'main_agent' to account 943986266082 (us-east-1)
Setting up AWS resources (ECR repository, execution roles)...
Using ECR repository from config: 943986266082.dkr.ecr.us-east-1.amazonaws.com/bedrock-agentcore-main_agent
Using execution role from config: arn:aws:iam::943986266082:role/AmazonBedrockAgentCoreSDKRuntime-us-east-1-dbcb276f38
Preparing CodeBuild project and uploading source...
Using CodeBuild role from config: arn:aws:iam::943986266082:role/AmazonBedrockAgentCoreSDKCodeBuild-us-east-1-dbcb276f38
Using .dockerignore with 44 patterns
Uploaded 

In [12]:
import time
from IPython.display import Markdown, display

status_response = agentcore_runtime.status()
status = status_response.endpoint['status']
end_status = ['READY', 'CREATE_FAILED', 'DELETE_FAILED', 'UPDATE_FAILED']
while status not in end_status:
    time.sleep(10)
    status_response = agentcore_runtime.status()
    status = status_response.endpoint['status']
    print(status)
status

Retrieved Bedrock AgentCore status for: main_agent


'READY'

In [13]:
invoke_response = agentcore_runtime.invoke({"prompt": "What's the BTC price today?",
     "thread_id": "test-thread-001",
     "actor_id": "test-actor-001"
     })
response_text = invoke_response['response'][0]

# Display with bigger text, BTC in orange, and line break
display(Markdown(f"""
<span style="font-size:30px;">**Agent Response:**</span><br><br>
<span style="font-size:25px;">{response_text.replace('BTC', '<span style="color:orange;">BTC</span>')}</span>
"""))


<span style="font-size:30px;">**Agent Response:**</span><br><br>
<span style="font-size:25px;">According to the latest data, the current Bitcoin (<span style="color:orange;">BTC</span>) price is $115,000 USD as of October 27, 2025.

This represents a significant increase from previous years. For context, Bitcoin first reached the $20,000 mark in late 2017, crossed $60,000 in 2021, and has now nearly doubled that value. If you're interested in more specific information about Bitcoin's price movements or trends, feel free to ask!</span>


In [ ]:
invoke_response = agentcore_runtime.invoke(
    {"prompt": "What's my previous question?",
     "thread_id": "test-thread-001",
     "actor_id": "test-actor-001"
     })
response_text = invoke_response['response'][0]

# Display with bigger text, BTC in orange, and line break
display(Markdown(f"""
<span style="font-size:30px;">**Agent Response:**</span><br><br>
<span style="font-size:25px;">{response_text.replace('BTC', '<span style="color:orange;">BTC</span>')}</span>
"""))


<span style="font-size:30px;">**Agent Response:**</span><br><br>
<span style="font-size:25px;">Your previous question was "What's the <span style="color:orange;">BTC</span> price today?" 

You asked about the current Bitcoin price, and I provided you with the information that Bitcoin is currently trading at $115,000 USD as of October 27, 2025, based on the data I retrieved.

Is there something specific about Bitcoin or your previous question that you'd like me to clarify?</span>


: 